<a href="https://colab.research.google.com/github/brenoakihiromorimoto/portf-lio/blob/main/app_home_match.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install

In [64]:
!pip install gradio
!pip install langchain
!pip install -U langchain-community
!pip install openai
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.8/526.8 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 38.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.1/106.1 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.6/67.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 

In [66]:
!pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.8 MB/s eta 0:00:00


#Run App

In [74]:
import random
import time
import gradio as gr
from langchain.memory import ConversationSummaryMemory, ConversationBufferMemory, CombinedMemory, ChatMessageHistory
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.chains import ConversationChain

import os

with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.ClearButton([msg, chatbot])
    #create variable to store Q/A:
    history = ChatMessageHistory()
    answer = []
    loader = CSVLoader("/content/df_home_match.csv")
    os.environ['OPENAI_API_KEY']="OPENAI_API_KEY"
    db = Chroma.from_documents(loader.load(), embedding=OpenAIEmbeddings())

    def respond(message, chat_history):
        #####WELCOME######
        if len(chat_history)==0:
            bot_message = """
            Hi, welcome to HomeMatch.
            I'm very glad you came here to find your dream house.
            I need you to answer just 5 questions, alright?
            """
        #####FIRST PART: Q&A######
        elif len(chat_history)==1:
            bot_message = "How big do you want your house to be?"
        elif len(chat_history)==2:
            bot_message = "What are 3 most important things for you in choosing this property?"
            answer.append(message)
        elif len(chat_history)==3:
            bot_message = "Which amenities would you like?"
            answer.append(message)
        elif len(chat_history)==4:
            bot_message = "Which transportation options are important to you?"
            answer.append(message)
        elif len(chat_history)==5:
            bot_message = "How urban do you want your neighborhood to be?"
            answer.append(message)
        elif len(chat_history)==6:
            answer.append(message)

        ##SECOND PART: SUGGESTIONS
            questions = [
                "How big do you want your house to be?",
                "What are 3 most important things for you in choosing this property?",
                "Which amenities would you like?",
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?"
                ]

            for i in range(len(questions)):
                history.add_ai_message(questions[i])
                history.add_user_message(answer[i])

            conversational_memory = ConversationBufferMemory(
                chat_memory = history,
                memory_key='questions_and_answers',
                input_key='input'
            )

            #Create summary memory conversation to input long-term-memory on live
            summary_memory = ConversationSummaryMemory(
                llm= ChatOpenAI(
                    model="gpt-3.5-turbo-0125",
                    temperature=1
                    ),
                memory_key='recommendation_summary',
                input_key='input',
                buffer=f'The human answered {len(questions)} personal questions. Use them to rate from 1 to 10, how much their buyer preferences.'
            )

            #Combine memories
            memory = CombinedMemory(memories=[conversational_memory, summary_memory])

            #
            RECOMMENDER_TEMPLATE = """
            The following is a friendly conversation between a human and an AI Home Recommender.
            The AI is follows human instructions and provides personalized home recommendation following preferences of user.

            Summary of Recommendations:
            {recommendation_summary}
            Personal Questions and Answers:
            {questions_and_answers}
            Human: {input}
            AI:"""

            PROMPT = PromptTemplate(
                input_variables=["recommendation_summary", "input", "questions_and_answers"],
                template=RECOMMENDER_TEMPLATE
                )
            # create a recommendation conversation chain that will let us ask AI for recommendations on top 3 homes
            recommender = ConversationChain(
                llm=ChatOpenAI(
                model="gpt-3.5-turbo-0125",
                temperature=1
                ),
                verbose=True,
                memory=memory,
                prompt=PROMPT
                )

            #We will focus just on top 1 suggestion.
            query = str(memory.memories[0].chat_memory.messages)

            docs_suggestion_prep = db.similarity_search(query, k=1)
            docs_suggestion = docs_suggestion_prep[0].page_content

            final_recommendation = f"""
             Emphasize important aspects of description below where it's alligned with buyer preferences,
             but you can't modify any factual information of home description:
             Follow the example format:
             ###EXAMPLE FORMAT###
             Neighborhood:
             Price:
             Bedrooms:
             Bathrooms:
             House Size:

             Description:

             Neighborhood Description:

             ###HOME DESCRIPTION###
            {docs_suggestion}
            """
            bot_message = recommender.predict(input=final_recommendation)

        chat_history.append((message, bot_message))
        time.sleep(2)
        return "", chat_history

    msg.submit(respond, [msg, chatbot], [msg, chatbot])

if __name__ == "__main__":
    demo.launch(debug=True, share=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://b632d1c527bfde4733.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)




> Entering new ConversationChain chain...
Prompt after formatting:

            The following is a friendly conversation between a human and an AI Home Recommender. 
            The AI is follows human instructions and provides personalized home recommendation following preferences of user. 

            Summary of Recommendations:
            The human answered 5 personal questions. Use them to rate from 1 to 10, how much their buyer preferences.
            Personal Questions and Answers:
            AI: How big do you want your house to be?
Human: mansion
AI: What are 3 most important things for you in choosing this property?
Human: 3 bedrooms, large garden
AI: Which amenities would you like?
Human: school nearby
AI: Which transportation options are important to you?
Human: bus
AI: How urban do you want your neighborhood to be?
Human: less urban
            Human: 
             Emphasize important aspects of description below where it's alligned with buyer preferences, 
          